# Turing Pattern Plot

This Jupyter _notebook_ computes and plots a [**Turing pattern**](https://en.wikipedia.org/wiki/Turing_pattern), named for [Alan Turing](https://en.wikipedia.org/wiki/Alan_Turing), one of the pioneers of computer science, who first described reaction-diffusion systems and realized their importance to analyze large-scale shape organisation in biological systems ([Turing, 1952](https://royalsocietypublishing.org/doi/10.1098/rstb.1952.0012)).

Think of your cat's stripes. They appear on a centimetre scale in the fur. However the individual cells that are responsible for their pigmentation are a million times smaller, and even the organized hair follicles may number more than a thousand across the width of the stripe. How do they know when to produce pigment, and when not to? This is a self-organized system, that needs no central authority to direct the cells: the patterns arise from the diffusion of molecules, their reaction, and decay.

A [reaction-diffusion mechanism](https://en.wikipedia.org/wiki/Reaction%E2%80%93diffusion_system) produces large-scale spatial patterns through the interaction of an _activator_ and an _inhibitor_. The activator promotes its own production (positive feedback) as well as that of the inhibitor, while the inhibitor suppresses the activator. Crucially, the inhibitor diffuses faster than the activator. The difference in diffusion rates destabilizes uniformity and may lead to periodic organization on a scale that is many orders of magnitude larger than the activator and inhibitor itself. Factors that sustain the production of the two are modelled through a "_feed_" term, whereas their removal or degradation is modeled by a "_kill_" term; together, these shape the dynamic balance between them, in time and space. The emerging patterns can be strikingly complex and evocative.

We simulate such a system below.

The first code cell sets up your computer and adds python librarties you might be missing. The cell below it computes an actual pattern and plots it. Put your cursor in a cell and hit `shift + enter` tu run it and advance, or `control + enter` to run it and stay in the cell.


In [ ]:

# ------------------------------------------------------------------------------------
# 
#      W E L C O M E
# 
#      RUN this cell first to install any required libraries
#      that might be missing on your computer.
#
# ------------------------------------------------------------------------------------

import subprocess
import sys

# Define the packages that are REQUIRED and EXTERNAL
REQUIRED_PACKAGES = ['numpy', 'matplotlib', 'tqdm'] 

# --- Robust Package Check and Installation for ALL External Dependencies ---

# Join the list of package names into a single string, separated by ", "
# and use an f-string to embed the resulting string into a message
print(f"Checking for required packages: {", ".join(REQUIRED_PACKAGES)}...")

for package in REQUIRED_PACKAGES:
    try:
        # Try a standard import check
        __import__(package)
        print(f"'{package}' found.")
        
    except ImportError:
        # If import fails, install the library
        print(f"'{package}' not found. Attempting installation...")
        
        try:
            # Use subprocess to run the command in the shell
            # This is equivalent to type e.g. "python -m pip install numpy" in the terminal
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            
            print(f"'{package}' installed successfully.")
            
        except Exception as e:
            # Catch installation errors (e.g., no internet, permission issues)
            print(f"ERROR: Could not install '{package}'. Please run 'python -m pip install {package}' manually.")
            # We can't proceed without these, so we raise an error
            raise e 

# --- Final Imports (Now that all packages are confirmed installed) ---
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm.notebook import tqdm 

print("All dependencies are ready!")


#   NOTE:
#   =====
#   If anything was actually installed, then RUN THIS CELL AGAIN. Some 
#   environments require a re-run of the cell after a fresh install.
#   and for the others it will simply do nothing.
#



&nbsp;

---

We compute the pattern and plot it here. You can edit the parameters, and re-run the cell to see what effect that might have ...


In [ ]:
# ------------------------------------------------------------------------------------
# 
#      RUN this cell to simulate a Turing pattern and create a plot from it.
#
# ------------------------------------------------------------------------------------


# --- I. Parameters  ---

# Simulation Settings
Nx = 414       # Nx x Ny grid
Ny = 256     
steps = 5000   # Total simulation steps 
dt = 1.0       # Time step
dx = 1.0       # Spatial step size
n_seeds = 100  # Number of randomly placed seed squares
CMAP = 'viridis' # some nice alternatives are berlin, bone, inferno, magma, twilight, and viridis

# Gray-Scott Parameters
# Note: The Gray-Scott Model is VERY sensitive to parameters. The list below is adapted 
#       from the ones in Jens Luebeck's Reaction Diffusion Simulator at:
#       https://github.com/jluebeck/ReactionDiffusionSimulator
#       ... all credit to him for finding them.

             #                     |  Coral:  |  Maze:   |  Worms:  |  Waves:  |
Du = 0.130   # Diffusivity of U    |  0.130   |  0.190   |  0.160   |  0.120   |
Dv = 0.060   # Diffusivity of V    |  0.060   |  0.050   |  0.080   |  0.080   |
F  = 0.060   # Feed Rate           |  0.060   |  0.060   |  0.054   |  0.020   |
k  = 0.062   # Kill Rate           |  0.062   |  0.062   |  0.064   |  0.050   |

# Note: The Waves pattern will need fewer steps to look good. Try 380 steps.
#       For the Worms pattern, try fewer seeds and more steps e.g. n_seeds=20, steps=50000 .

# --- II. Functions ---

def laplacian_operator_standard(M):
    """
    Computes the 2D Laplacian operator with periodic boundaries 
    using the standard NumPy roll method.
    """
    neighbors = (
        np.roll(M, 1, axis=0) + np.roll(M, -1, axis=0) +
        np.roll(M, 1, axis=1) + np.roll(M, -1, axis=1)
    )
    return neighbors - 4 * M

def simulate_pattern(U, V, steps, Du, Dv, F, k, dt):
    """Run Turing pattern simulation ."""
    for i in tqdm(range(steps), desc="Simulating pattern", ncols=600):
        Lu = laplacian_operator_standard(U)
        Lv = laplacian_operator_standard(V)
        uvv = U * V * V
        U += (Du * Lu - uvv + F * (1.0 - U)) * dt
        V += (Dv * Lv + uvv - (F + k) * V) * dt
        np.clip(U, 0, 1, out=U)
        np.clip(V, 0, 1, out=V)
    return U, V


# --- III. Initialization ---

w_seeds = 4
sd = 7/13

# random normal coordinates, skewed toward top-left
seed_x = (np.abs(np.random.normal(0, sd, n_seeds)) * (Nx - w_seeds)).astype(int) + w_seeds
seed_y = (np.abs(np.random.normal(0, sd, n_seeds)) * (Ny - w_seeds)).astype(int) + w_seeds
seed_w = np.full(n_seeds, w_seeds, dtype=int)

initial_seeds = list(zip(seed_x, seed_y, seed_w))


# --- IV. Setup ---

# Initialization Matrices (Note: x, y swapped. Not a typo.)
U = np.ones((Ny, Nx)) 
V = np.zeros((Ny, Nx)) 

# Apply initial conditions for each square
for y, x, w in initial_seeds:
    x_start, x_end = max(0, x - w), min(Ny, x + w)
    y_start, y_end = max(0, y - w), min(Nx, y + w)
    
    # In the patch: V is high (V=1.0), U is low (U=0.5)
    U[x_start:x_end, y_start:y_end] = 0.50
    V[x_start:x_end, y_start:y_end] = 0.25


# --- V. Run the simulation ---
U_final, V_final = simulate_pattern(
    U, V,
    steps,
    Du, Dv, F, k, dt
)


# --- VI. Plot ---
scale = 14
plt.figure(figsize=(scale, scale*(Nx/Ny)))
plt.imshow(V_final, cmap=CMAP, interpolation='bilinear')
plt.title(f'Final Turing Pattern ({steps} steps)', fontsize=14)
plt.axis('off')
plt.show()



&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

That is all.
<!-- Before committing and distributing the final notebook, at the commandline issue:
     jupyter nbconvert --clear-output --inplace src/py/turing_pattern.ipynb
-->